In [1]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime

## Group Assignment
### Team Number: XX
### Team Member Names:
### Team Strategy Chosen: __________(Market Beat or Market Meet)

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.


In [5]:
tickers_file_name = "Tickers_Example.csv"
tickers_file_original = pd.read_csv(tickers_file_name , header = None, names=['Ticker'])

start_date = "2023-10-02"
end_date = "2024-09-30"
drop_tickers = []

for tick in tickers_file_original['Ticker']:    

    try:

        ticker = yf.Ticker(tick) 

        if ticker.info['quoteType'] != 'EQUITY':
            drop_tickers.append(tick)
            continue
                
        if ticker.fast_info.get('currency') not in ['USD', 'CAD']:
            drop_tickers.append(tick)
            continue

        history = ticker.history(start = start_date, end = end_date)

        trading_days = history['Volume'].resample('ME').count()
        volume = history['Volume'].resample('ME').sum()
        trading_days = trading_days[trading_days>=18]
        volume = volume[volume.index==trading_days.index]
        monthly_volume = volume.mean()

        if monthly_volume<100000: 
            drop_tickers.append(tick)

    except:
            drop_tickers.append(tick)
            pass
    
tickers_file_original.drop(tickers_file_original[tickers_file_original["Ticker"].isin(drop_tickers)].index, inplace=True)
tickers_file_original = tickers_file_original.drop_duplicates(subset=['Ticker'], keep='last')
tickers_file_original = tickers_file_original.reset_index(drop=True)
tickers_file_original.index = tickers_file_original.index + 1
# print(tickers_file_original)

In [7]:
tickers_file_original

,Ticker
1,AAPL
2,ABBV
3,ABT
4,ACN
5,AIG
6,AMZN
7,AXP
8,BA
9,BAC
10,BB.TO


In [41]:
# take exchange rates from yfinance
exchange = 'USDCAD=X'
exchange_ticker = yf.Ticker(exchange)
exchange_rate = exchange_ticker.history(interval = '1mo', start=start_date, end=end_date)["Close"]
exchange_rate = pd.DataFrame(exchange_rate)
exchange_rate.index = exchange_rate.index.strftime('%Y-%m-%d')

In [89]:
# Define stock list and market index
stock_list = tickers_file_original.Ticker.tolist()
market_indices = ["^GSPC", 'XIU.TO']  

# Fetch historical data for each stock and market indices
current_data = yf.download(stock_list + market_indices, interval = '1mo', start=start_date, end=end_date)["Close"]
current_data.index = current_data.index.strftime('%Y-%m-%d')

# convert all stock prices to CAD, if needed

for ticker in stock_list:
    ticker_name = yf.Ticker(ticker)
    country = ticker_name.fast_info["currency"]
    if country == 'USD':
        current_data[ticker] = exchange_rate['Close'] * current_data[ticker]
    elif country == 'CAD':
        continue
    else:
        raise ValueError(f"Unsupported currency: {country}")

current_data['^GSPC'] = current_data['^GSPC'] * exchange_rate['Close']
current_data['XIU.TO'] = current_data['XIU.TO'] * exchange_rate['Close']

display(current_data)

[*********************100%***********************]  39 of 39 completed


Ticker,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BB.TO,...,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB,XIU.TO,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
2023-11-01,258.145844,193.510856,141.732196,452.743941,89.437103,198.539226,232.079850,314.789808,41.436519,4.98,...,98.849998,24.280001,82.739998,207.535953,751.497247,306.146440,206.041022,51.805841,41.817046,6207.731267
2023-12-01,255.314030,205.505717,145.963826,465.341753,89.843274,201.487636,248.431568,345.661229,44.649784,4.70,...,103.160004,23.580000,85.620003,226.047014,698.151824,325.716674,208.502696,57.393606,42.461722,6325.271627
2024-01-01,247.127345,220.323944,151.640240,487.661076,93.155221,207.994384,269.025738,282.829473,45.579180,3.77,...,107.629997,24.080000,81.669998,214.588018,685.818596,326.907665,190.170118,55.670664,43.126672,6493.994755
2024-02-01,245.413323,239.031902,161.083466,508.857565,98.966401,239.995893,297.917515,276.600844,46.869533,3.80,...,103.690002,23.670000,81.489998,227.192320,670.185437,344.447616,201.300016,56.971191,44.181188,6919.460915
2024-03-01,232.166759,246.545190,153.884274,469.275244,105.834357,244.216481,308.269484,261.289160,51.339884,3.71,...,104.500000,21.670000,81.750000,235.862920,669.774325,332.964606,201.230157,60.519329,45.694124,7113.864365
2024-04-01,232.784906,222.275210,144.826023,411.244678,102.923915,239.167252,319.841781,229.381890,50.580455,3.85,...,96.650002,22.110001,81.669998,241.107921,661.058302,324.119465,201.556488,55.527804,45.127444,6882.126435
2024-05-01,263.061443,220.629537,139.829646,386.265888,107.851770,241.428149,328.399200,243.029092,54.719519,3.80,...,80.660004,22.410000,76.199997,266.838026,677.829626,318.574601,190.102077,55.485781,46.016939,7221.374947
2024-06-01,288.380895,234.845188,142.273576,415.428974,101.649404,264.597898,317.038262,249.208083,54.453084,3.42,...,90.410004,20.709999,75.199997,266.350473,697.278801,309.795182,187.375027,54.357241,45.238369,7476.489141
2024-07-01,307.571932,256.660808,146.722672,457.895487,109.730390,258.959825,350.450283,263.973395,55.827742,3.33,...,84.559998,22.290001,81.529999,282.268705,797.958548,341.711189,180.557236,62.157009,48.515149,7648.164643


In [69]:
sp500 = yf.Ticker("^GSPC").history(start=start_date, end=end_date)['Close']
sp500.index = sp500.index.date

tsx60 = yf.Ticker("XIU.TO").history(start=start_date, end=end_date)['Close']
tsx60.index = tsx60.index.date

total_investment = 1000000
sp500val = (total_investment / sp500.iloc[0]) * sp500.iloc[-1]
sp500val = round(sp500val,2)
# sp500val

tsx60val = (total_investment / tsx60.iloc[0]) * tsx60.iloc[-1]
tsx60val = round(tsx60val,2)
# tsx60val

average = (sp500val + tsx60val) / 2
average = round(average,2)
average

1311957.2

In [73]:
sp500 = current_data['^GSPC']
#sp500.index = sp500.index.date
sp500_return = sp500.pct_change().dropna()
tsx60 = current_data['XIU.TO'] 
#tsx60.index = tsx60.index.date
tsx60_return = tsx60.pct_change().dropna()

common_dates = sp500_return.index.intersection(tsx60_return.index)

average_df = pd.DataFrame(data = {"% Return" : (sp500_return.loc[common_dates] + tsx60_return.loc[common_dates])/2}, index = common_dates)
#average_df

In [75]:
metrics = pd.DataFrame(index=stocks, columns=['SP-beta','SP-corr','TSX-beta','TSX-corr', 'S&P500', 'TSX60'])

url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_df = pd.read_html(url)[0]
sp500companies = sp500_df.Symbol.tolist()

tsx60companies = ["RY.TO", "TD.TO", "SHOP.TO", "ENB.TO", "CNR.TO", "SU.TO", "BN.TO", "CSU.TO", 
    "MFC.TO", "TRP.TO", "BMO.TO", "BNS.TO", "ATD.TO", "ABX.TO", "BCE.TO", "CM.TO", 
    "CP.TO", "NTR.TO", "TRI.TO", "WCN.TO", "MG.TO", "L.TO", "IFC.TO", "WN.TO", 
    "SLF.TO", "FNV.TO", "TECK.B.TO", "DOL.TO", "OTEX.TO", "FM.TO", "IMO.TO", 
    "CVE.TO", "PPL.TO", "CNQ.TO", "FTS.TO", "H.TO", "RCI.B.TO", "T.TO", "AQN.TO", 
    "SAP.TO", "MRU.TO", "QSR.TO", "WPM.TO", "POW.TO", "EMA.TO", "AEM.TO", 
    "GIL.TO", "GIB.A.TO", "TOU.TO", "BEP.UN.TO", "BIP.UN.TO", "CTC.A.TO", 
    "CCL.B.TO", "K.TO", "WSP.TO", "IFP.TO"]

def in_sp500(ticker, sp500companies):
    if ticker in sp500companies:
        return 5
    else:
        return 0
    
def in_tsx60(ticker, tsx60companies):
    if ticker in tsx60companies:
        return 5
    else: 
        return 0

In [97]:
sp500_returns = current_data['^GSPC'].pct_change().dropna()
tsx60_returns = current_data['XIU.TO'].pct_change().dropna()
stocks = {key: 0 for key in tickers_file_original['Ticker'].values}

for i in stock_list:
    
    stock_returns = current_data[i].pct_change().dropna()
    
    covariance = np.cov(stock_returns, sp500_returns)[0, 1]
    variance = np.var(sp500_returns)
    beta_sp500 = covariance / variance
    
    correlation_sp500 = np.corrcoef(stock_returns, sp500_returns)[0, 1]
    
    covariance_tsx = np.cov(stock_returns, tsx60_returns)[0, 1]
    variance_tsx = np.var(tsx60_returns)
    beta_tsx60 = covariance_tsx / variance_tsx
    
    correlation_tsx60 = np.corrcoef(stock_returns, tsx60_returns)[0, 1]
    
    metrics.loc[i, 'SP-corr'] = correlation_sp500
    metrics.loc[i, 'SP-beta'] = beta_sp500
    metrics.loc[i, 'TSX-corr'] = correlation_tsx60
    metrics.loc[i, 'TSX-beta'] = beta_tsx60

    stocks[i] = round(100 * (correlation_sp500 + beta_tsx60 + beta_sp500 + correlation_tsx60),2)

metrics

,SP-beta,SP-corr,TSX-beta,TSX-corr,S&P500,TSX60
AAPL,0.618093,0.256957,-0.004381,-0.001876,NaN,NaN
ABBV,1.523501,0.647057,0.954608,0.417591,NaN,NaN
ABT,0.663492,0.390891,0.171233,0.103905,NaN,NaN
ACN,1.272068,0.426654,0.923442,0.319008,NaN,NaN
AIG,1.135495,0.505683,1.620987,0.74353,NaN,NaN
AMZN,1.880512,0.721866,0.12816,0.050671,NaN,NaN
AXP,0.492503,0.260498,1.323727,0.721142,NaN,NaN
BA,1.643943,0.41507,1.142112,0.297009,NaN,NaN
BAC,0.811234,0.444706,0.673471,0.380253,NaN,NaN
BB.TO,-0.478366,-0.137371,0.676436,0.200073,NaN,NaN


In [117]:
beta_threshold = 0.1  
penalty_weight = 20  

ranked_stocks = {}
for ticker, score in stocks.items():
    sp_beta = metrics.loc[ticker, 'SP-beta']
    tsx_beta = metrics.loc[ticker, 'TSX-beta']
    in_index = metrics.loc[ticker, 'S&P500'] + metrics.loc[ticker, 'TSX60']
    
    sp_penalty = abs(sp_beta - 1) * penalty_weight
    tsx_penalty = abs(tsx_beta - 1) * penalty_weight

    adjusted_score = score - (sp_penalty + tsx_penalty) + in_index
    ranked_stocks[ticker] = adjusted_score

stocks = dict(sorted(stocks.items(), key=lambda item: item[1], reverse=True))
stocks = list(stocks.items())[:24]
stocks = dict(stocks)
stocks

{'AAPL': nan,
 'ABBV': nan,
 'ABT': nan,
 'ACN': nan,
 'AIG': nan,
 'AMZN': nan,
 'AXP': nan,
 'BA': nan,
 'BAC': nan,
 'BB.TO': nan,
 'BIIB': nan,
 'BK': nan,
 'BLK': nan,
 'BMY': nan,
 'C': nan,
 'CAT': nan,
 'CL': nan,
 'KO': nan,
 'LLY': nan,
 'LMT': nan,
 'MO': nan,
 'MRK': nan,
 'PEP': nan,
 'PFE': nan}

In [101]:
!pip install cvxpy

In [103]:
import cvxpy as cp

In [105]:

def give_weight(num_stocks):
    stocks_now = list(stocks.keys())[:num_stocks]
    prices = np.array(current_data[stocks_now].iloc[0])
    last_values = np.array(current_data[stocks_now].iloc[-1])
    w = cp.Variable(num_stocks)
    my_value = cp.sum(cp.multiply(last_values, w * 1000000 / prices))
    min_weight = (100 / (2 * num_stocks)) / 100
    objective = cp.Minimize(my_value - sp500val)
    constraints = [
        cp.sum(w) == 1,           
        w >= min_weight,                    
        w <= 0.15                      
    ]
    problem = cp.Problem(objective, constraints)
    problem.solve()
    return w.value


In [107]:
def get_value(weights,i):
    last_row = current_data[list(stocks.keys())[:i]].iloc[-1]
    first_row = current_data[list(stocks.keys())[:i]].iloc[0]
    return np.sum(np.multiply(last_row,np.divide(weights*1000000,first_row)))

In [109]:
def stocks_ith_portfolio(numStocks):
    weight_now = give_weight(numStocks)
    curr_val = get_value(weight_now, numStocks)
    return curr_val

In [111]:
diff = sp500val
close = 0
best_weights = []
best = 0
for i in range(12,25):
    curr_val = stocks_ith_portfolio(i)
    print(curr_val)
    if (abs(average - curr_val) < diff):
        diff = abs(curr_val - average)
        best = i
        best_weights = weight_now
        close = curr_val

992920.2593031158


NameError: name 'weight_now' is not defined

In [113]:
ranked_stocks = stocks

stockslist = list(ranked_stocks.keys())
weightlist = best_weight
weights = dict(zip(stockslist, weightlist))
total_money = 1000000
final_allocation = {}
for stock, weight in weights.items():
    if weight > 0:
        allocation = total_money * weight  
        ticker = yf.Ticker(stock)
        price = ticker.fast_info["lastPrice"]  # need to make sure "end_date" is the most recent trading day
        '''
        currency = ticker.fast_info["currency"]     
        if currency == 'USD':
            exchange = 'USDCAD=X'
            exchange_ticker = yf.Ticker(exchange)
            exchange_rate = exchange_ticker.fast_info["lastPrice"]
            price *= exchange_rate
        elif currency == 'CAD':
            continue
        '''
        num_shares = (allocation / price)
        # considering transaction costs
        if num_shares > 3950:
            num_shares = (allocation - 3.95)/price
            transaction_cost = 3.95
        else:
            num_shares = allocation/(price+0.001)
            transaction_cost = num_shares*0.001
        total_cost = num_shares * price
        #transaction_cost = total_cost * 0.01
        final_allocation[stock] = {
            "Number of Shares": num_shares,
            "Transaction Cost": transaction_cost,
            "Total Money Spent": total_cost + transaction_cost,
            "Cumulative Transaction Cost": np.round(sum(stock["Transaction Cost"] for stock in final_allocation.values()), 2),
            "Cumulative Total": np.round(total_cost + transaction_cost+ sum(stock["Total Money Spent"] for stock in final_allocation.values()), 2)
        }
final = pd.DataFrame(final_allocation)

print(f"Total Money Spent: ${sum(a['Total Money Spent'] for a in final_allocation.values()):.2f}")
print(f"Total Cost: ${sum(a['Transaction Cost'] for a in final_allocation.values()):.2f}")
display(final)

NameError: name 'best_weight' is not defined

In [ ]:
# put everything into the final portfolio

Portfolio_Final = pd.DataFrame({"Tickers": [], 
                   "Price":[], 
                   "Currency":[], 
                   "Shares":[], 
                   "Value":[], 
                   "Weight":[],
                   "Transactional cost": [],
                   "Cumulative Value": [],
                   "Cumulative Transactional Cost": []})
cumulative_value = 0
cumulative_transaction_cost = 0
exchange_rate_ticker = "USDCAD=X"
latest_exchange_rate = yf.Ticker(exchange_rate_ticker).fast_info['lastPrice']

for stock, weight in weights.items():
    if weight > 0:
        allocation = total_money * weight  
        ticker = yf.Ticker(stock)
        price = ticker.fast_info["lastPrice"]  # need to make sure "end_date" is the most recent trading day
        country = ticker.fast_info['currency']
        if country == "USD:
            price *= latest_exchange_rate
        num_shares = (allocation / price)
        
        # considering transaction costs
        if num_shares > 3950:
            num_shares = (allocation - 3.95)/price
            transaction_cost = 3.95
        else:
            num_shares = allocation/(price+0.001)
            transaction_cost = num_shares*0.001
        total_cost = num_shares * price

        value = price * num_shares
        cumulative_value += value
        cumulative_transaction_cost += transaction_cost

        new_row = {"Tickers": stock, 
                   "Price": price, 
                   "Currency": "CAD", 
                   "Shares": num_shares, 
                   "Value": value, 
                   "Weight": weight,
                   "Transactional cost": transaction_cost,
                   "Cumulative Value": cumulative_value, 
                   "Cumulative Transactional Cost": cumulative_transaction_cost}
        Portfolio_Final.loc[len(Portfolio_Final)] = new_row 

Portfolio_Final.index = range(1, len(Portfolio_Final) + 1)

Portfolio_Final

In [ ]:
Stocks_Final = pd.DataFrame({"Tickers": Portfolio_Final['Tickers'], 
                             "Shares": Portfolio_Final['Shares']})
display(Stocks_Final)
# output last dataframe as a CSV file
Stocks_Final.to_csv()

In [53]:
diff = sp500val
close = 0
for i in range(12,25):
    weight_now = give_weight(i)
    last_row = current_data[list(stocks.keys())[:i]].iloc[-1]
    first_row = current_data[list(stocks.keys())[:i]].iloc[0]
    curr_val = np.sum(np.multiply(last_row,np.divide(weight_now*1000000,first_row)))
    if (abs(curr_val - sp500val) < diff):
        diff = abs(curr_val - sp500val)
        close = curr_val
print(close)
print(diff)

1177361.3213981315
160709.54860186856


In [54]:
print(diff/close)

0.13649976917113596


In [47]:
sp500val

np.float64(1338070.87)

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.

In [ ]:
Abbas, Gurmehar, Elaine